# s3 데이터셋 적재를 위한 테스트 데이터 필터링
- 기간: 220601 ~ 220607

In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import copy
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')
pd.set_option("display.max_columns", None)

## 1. 데이터셋 필터링

In [2]:
# 현재 작업 디렉토리 가져오기
original_directory = os.getcwd()
print(f"원래 디렉토리: {original_directory}")

원래 디렉토리: c:\Users\aryij\Documents\DataStudy\epoch\final-project


In [19]:
# poi 데이터는 날짜 컬럼 없음
# poi_df = pd.read_csv(os.path.join("data","aihub", "metadata", "tn_poi_master_POIMaster_all.csv"))
poi_df = pd.read_parquet(os.path.join("data","aihub", "metadata", "tn_poi_master_POIMaster_all.parquet"))
poi_df.head(3)

,POI_ID,POI_NM,BRNO,SGG_CD,ROAD_NM_ADDR,LOTNO_ADDR,ASORT_LCLASDC,ASORT_MLSFCDC,ASORT_SDASDC,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD
0,POI0100000000001X,장안평,,1123010600,None,서울특별시 동대문구 장안동 566 대,NaN,NaN,NaN,127.068515,37.563459,,1123010600105660000
1,POI0100000000002Y,육연평야,,4136025322,None,경기도 남양주시 진접읍 내각리 445-2답,NaN,NaN,NaN,127.168214,37.697096,,4136025322104450002
2,POI0100000000003Z,두평야,,4157025321,None,경기도 김포시 고촌읍 향산리 401 대,NaN,NaN,NaN,126.744188,37.624167,,4157025321104010000


In [3]:
path_parquet_22 = os.path.join("data", "aihub", "2022", "total_combined", "gps_data_parquet", "total_gps_2022.parquet")
path_parquet_23 = os.path.join("data", "aihub", "2023", "total_combined", "gps_data_parquet", "total_gps_2023.parquet")
gps_22 = pd.read_parquet(path_parquet_22, engine="pyarrow")
gps_23 = pd.read_parquet(path_parquet_23, engine="pyarrow")

In [4]:
gps_22.head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
0,a9c1a065c89da3b7,126.702718,37.500779,a_a000011,capital,2022-08-06,12:33:00
1,a9c1a065c89da3b7,126.703540,37.506400,a_a000011,capital,2022-08-06,12:34:00
2,a9c1a065c89da3b7,126.706564,37.506687,a_a000011,capital,2022-08-06,12:35:00


In [5]:
gps_23.head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
0,e000004_900009,127.054289,37.209919,e_e000004,capital,2023-04-30,13:30:00
1,e000004_900009,127.053911,37.208667,e_e000004,capital,2023-04-30,13:30:00
2,e000004_900009,127.053411,37.207996,e_e000004,capital,2023-04-30,13:31:00


In [6]:
# gps 데이터 언제부터 있는지 확인
print(f'2022년 가장 빠른 일자: {np.min(gps_22["DT_YMD"])}')
print(f'2022년 가장 늦은 일자: {np.max(gps_22["DT_YMD"])}')

2022년 가장 빠른 일자: 2022-08-03
2022년 가장 늦은 일자: 2022-11-14


In [7]:
print(f'2023년 가장 빠른 일자: {np.min(gps_23["DT_YMD"])}')
print(f'2023년 가장 늦은 일자: {np.max(gps_23["DT_YMD"])}')

2023년 가장 빠른 일자: 2023-04-28
2023년 가장 늦은 일자: 2023-09-30


In [8]:
region_22 = os.listdir(os.path.join("data", "aihub", "2022", "total_combined", "region_data"))
region_23 = os.listdir(os.path.join("data", "aihub", "2023", "total_combined", "region_data"))

In [12]:
def check_date(directory):
    """
    특정 디렉터리 내 CSV 파일에서 "YMD"라는 글자를 포함한 컬럼의 최소값과 최대값을 출력하는 함수.

    Parameters:
        directory (str): 검색할 디렉터리 경로.
    """
    # 디렉터리 내용 가져오기
    all_files = os.listdir(directory)
    
    # .csv 파일 필터링
    csv_files = [f for f in all_files if f.endswith('.csv') and os.path.isfile(os.path.join(directory, f))]
    
    for csv_file in csv_files:
        # CSV 파일 읽기
        df = pd.read_csv(os.path.join(directory, csv_file))

        # "YMD"라는 글자를 포함한 컬럼 찾기
        if csv_file == 'tn_traveller_master_여행객 Master':
            ymd_columns = ["TRAVEL_STATUS_START_YMD", "TRAVEL_STATUS_END_YMD"]
        else:
            ymd_columns = [col for col in df.columns if "YMD" in col]

        for col in ymd_columns:
            # "YMD" 컬럼에서 최소값 및 최대값 찾기 (errors='coerce'로 에러 방지)
            dates = pd.to_datetime(df[col], errors='coerce')

            # 유효한 날짜만 남기기
            valid_dates = dates.dropna()

            if not valid_dates.empty:
                min_date = valid_dates.min()
                max_date = valid_dates.max()

                print(f'{csv_file}, {col} 최소값: {min_date}')
                print(f'{csv_file}, {col} 최대값: {max_date}\n')
            else:
                print(f'{csv_file}, {col}에는 유효한 날짜가 없습니다.\n')


In [13]:
check_date(os.path.join("data", "aihub", "2022", "total_combined", "region_data"))

tn_activity_consume_his_활동소비내역.csv, PAYMENT_DT_YMD 최소값: 2022-06-01 00:00:00
tn_activity_consume_his_활동소비내역.csv, PAYMENT_DT_YMD 최대값: 2023-02-28 00:00:00

tn_adv_consume_his_사전소비내역.csv, PAYMENT_DT_YMD 최소값: 2022-01-01 00:00:00
tn_adv_consume_his_사전소비내역.csv, PAYMENT_DT_YMD 최대값: 2022-11-25 00:00:00

tn_lodge_consume_his_숙박소비내역.csv, CHK_YMD 최소값: 2022-08-03 00:00:00
tn_lodge_consume_his_숙박소비내역.csv, CHK_YMD 최대값: 2022-11-13 00:00:00

tn_lodge_consume_his_숙박소비내역.csv, CHK_OUT_DT_YMD 최소값: 2022-08-04 00:00:00
tn_lodge_consume_his_숙박소비내역.csv, CHK_OUT_DT_YMD 최대값: 2022-11-14 00:00:00

tn_lodge_consume_his_숙박소비내역.csv, PAYMENT_DT_YMD 최소값: 2022-01-01 00:00:00
tn_lodge_consume_his_숙박소비내역.csv, PAYMENT_DT_YMD 최대값: 2022-11-12 00:00:00

tn_move_his_이동내역.csv, START_DT_YMD 최소값: 2022-08-03 00:00:00
tn_move_his_이동내역.csv, START_DT_YMD 최대값: 2022-11-13 00:00:00

tn_move_his_이동내역.csv, END_DT_YMD 최소값: 2022-08-03 00:00:00
tn_move_his_이동내역.csv, END_DT_YMD 최대값: 2022-11-13 00:00:00

tn_mvmn_consume_his_이동수단소비내역.csv, PAYME

In [14]:
check_date(os.path.join("data", "aihub", "2023", "total_combined", "region_data"))

tn_activity_consume_his_활동소비내역.csv, PAYMENT_DT_YMD 최소값: 2023-04-04 00:00:00
tn_activity_consume_his_활동소비내역.csv, PAYMENT_DT_YMD 최대값: 2023-09-30 00:00:00

tn_adv_consume_his_사전소비내역.csv, PAYMENT_DT_YMD 최소값: 2023-03-14 00:00:00
tn_adv_consume_his_사전소비내역.csv, PAYMENT_DT_YMD 최대값: 2023-09-28 00:00:00

tn_lodge_consume_his_숙박소비내역.csv, CHK_YMD에는 유효한 날짜가 없습니다.

tn_lodge_consume_his_숙박소비내역.csv, CHK_OUT_DT_YMD에는 유효한 날짜가 없습니다.

tn_lodge_consume_his_숙박소비내역.csv, PAYMENT_DT_YMD 최소값: 2023-01-01 00:00:00
tn_lodge_consume_his_숙박소비내역.csv, PAYMENT_DT_YMD 최대값: 2023-09-17 00:00:00

tn_move_his_이동내역.csv, START_DT_YMD 최소값: 2023-04-28 00:00:00
tn_move_his_이동내역.csv, START_DT_YMD 최대값: 2023-09-30 00:00:00

tn_move_his_이동내역.csv, END_DT_YMD 최소값: 2023-04-28 00:00:00
tn_move_his_이동내역.csv, END_DT_YMD 최대값: 2023-09-30 00:00:00

tn_mvmn_consume_his_이동수단소비내역.csv, PAYMENT_DT_YMD 최소값: 2023-01-03 00:00:00
tn_mvmn_consume_his_이동수단소비내역.csv, PAYMENT_DT_YMD 최대값: 2023-09-28 00:00:00

tn_tour_photo_관광사진.csv, PHOTO_FILE_DT_YMD 최소값: 

In [ ]:
def filter_by_date_region(dir1="aihub", dir2="2022", start_date="2022-06-01", end_date="2022-06-06"):
    
    # 기본 경로 설정 (data\aihub\2022)
    base_path = os.path.join("data", dir1, dir2, "total_combined", "region_data")

    dataset_list = os.listdir(base_path)

    region_dfs = {}
    # region_gps_dfs = {}

    # gps_data, region_data 폴더 내부를 for loop으로 들어간다
    for dataset in dataset_list:
        dataset_path = os.path.join(base_path, dataset)
        # dataset_list = os.listdir(f_path)

        # for dataset in dataset_list:
        #     file_path = os.path.join(folder_path, dataset)

        if not dataset.endswith('.csv'):
            print(f"파일 스킵: {dataset} (CSV 파일 아님)")
            continue

        try:
            # 파일 읽기 시도
            df = pd.read_csv(dataset_path)
            df_name = dataset.replace(".csv", "")
            
            # 파일에 데이터가 없는 경우 스킵
            if df.empty:
                print(f"파일 스킵: {dataset} (데이터 없음)")
                continue

            # 데이터프레임에서 "YMD"로 끝나는 컬럼 찾는다 (START_DATE_YMD는 제외)
            ymd_columns = [col for col in df.columns if col.endswith('YMD') and "START" not in col]

            if not ymd_columns:
                print(f"YMD 컬럼 없음: {dataset}\n")

                # if folder == "gps_data":
                #     region_gps_dfs[df_name] = df
                # else:
                region_dfs[df_name] = df
                
                continue
            
            if len(ymd_columns) > 1:
                ymd_columns = [ymd_columns[-1]]

            print(f"데이터셋: {dataset}\nYMD 컬럼: {ymd_columns}\n")

            # YMD 컬럼 처리
            for ymd_col in ymd_columns:
                df[ymd_col] = pd.to_datetime(df[ymd_col], errors="coerce")

            # 날짜 필터링 (ymd_columns 중 첫 번째 컬럼)
            filtered_df = df[(df[ymd_columns[0]] >= start_date) & (df[ymd_columns[0]] <= end_date)]

            # df_name = dataset.replace(".csv", "")

            # if folder == "gps_data":
            #     region_gps_dfs[df_name] = filtered_df
            # else:
            region_dfs[df_name] = filtered_df

        except pd.errors.EmptyDataError:
            print(f"파일 스킵: {dataset} (비어 있는 파일)")
        except Exception as e:
            print(f"파일 처리 오류: {dataset}: {e}")

    print(f"===================== {dir2} 데이터 필터링 완료 ({start_date} ~ {end_date}) =====================\n")

    # print(f"===================== 데이터 필터링 완료 (시작 날짜 {start_date}) =====================")
    return region_dfs


In [ ]:
# gps 데이터 필터링 함수
def filter_by_date_gps(df, ymd_col="DT_YMD", start_date="2022-06-01", end_date="2022-06-06"):

    df[ymd_col] = pd.to_datetime(df[ymd_col], errors="coerce")
    filtered_df = df[(df[ymd_col] >= start_date) & (df[ymd_col] <= end_date)]

    return filtered_df

In [ ]:
filtered_dfs_22 = filter_by_date_region("aihub", "2022", "2022-08-10", "2022-09-09")

YMD 컬럼 없음: tc_codea_코드A.csv

YMD 컬럼 없음: tc_codeb_코드B.csv

YMD 컬럼 없음: tc_sgg_시군구코드.csv

데이터셋: tn_activity_consume_his_활동소비내역.csv
YMD 컬럼: ['PAYMENT_DT_YMD']

YMD 컬럼 없음: tn_activity_his_활동내역.csv

데이터셋: tn_adv_consume_his_사전소비내역.csv
YMD 컬럼: ['PAYMENT_DT_YMD']

YMD 컬럼 없음: tn_companion_info_동반자정보.csv

데이터셋: tn_lodge_consume_his_숙박소비내역.csv
YMD 컬럼: ['PAYMENT_DT_YMD']

데이터셋: tn_move_his_이동내역.csv
YMD 컬럼: ['END_DT_YMD']

데이터셋: tn_mvmn_consume_his_이동수단소비내역.csv
YMD 컬럼: ['PAYMENT_DT_YMD']

데이터셋: tn_tour_photo_관광사진.csv
YMD 컬럼: ['PHOTO_FILE_DT_YMD']

데이터셋: tn_traveller_master_여행객 Master.csv
YMD 컬럼: ['TRAVEL_STATUS_END_YMD']

데이터셋: tn_travel_여행.csv
YMD 컬럼: ['TRAVEL_END_YMD']

데이터셋: tn_visit_area_info_방문지정보.csv
YMD 컬럼: ['VISIT_END_YMD']

===================== 2022 데이터 필터링 완료 (2022-08-10 ~ 2022-09-09) =====================



In [43]:
filtered_dfs_22["tn_mvmn_consume_his_이동수단소비내역"].head(2)

,TRAVEL_ID,MVMN_SE,PAYMENT_SE,PAYMENT_SEQ,MVMN_SE_NM,PAYMENT_NUM,BRNO,STORE_NM,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,REGION,PAYMENT_DT_YMD,PAYMENT_DT_MIN
12,a_a001405,2,렌탈비,3,렌터카(승용/승합/버스 등등),2,6.168191e+09,주식회사 쏘카,2022-08-24 22:41:00,1.0,44070.0,NaN,capital,2022-08-24,22:41:00
41,a_a001981,1,주차비,12,자가용(승용/승합/트럭 등등),5,1.148101e+09,고속버스터미널,2022-09-09 19:12:00,1.0,11000.0,주차비,capital,2022-09-09,19:12:00


In [ ]:
filtered_dfs_22["tn_move_his_이동내역"].head(3)

,TRAVEL_ID,TRIP_ID,START_VISIT_AREA_ID,END_VISIT_AREA_ID,MVMN_CD_1,MVMN_CD_2,REGION,START_DT_YMD,START_DT_MIN,END_DT_YMD,END_DT_MIN
0,a_a001352,2208260004,NaN,2.208260e+09,15.0,NaN,capital,NaN,NaN,2022-08-26,12:00:00
10,a_a000221,2208180005,NaN,2.208180e+09,1.0,NaN,capital,NaN,NaN,2022-08-18,12:30:00
22,a_a001129,2208280002,NaN,2.208280e+09,5.0,15.0,capital,NaN,NaN,2022-08-28,14:30:00


In [51]:
filtered_gps_dfs_22 = filter_by_date_gps(gps_22, "DT_YMD", "2022-08-10", "2022-09-09")

In [52]:
# 필터링된 gps 데이터 확인
filtered_gps_dfs_22

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
2052,6037b804983c662d,127.018340,37.280997,a_a000012,capital,2022-08-10,00:21:00
2053,6037b804983c662d,127.018102,37.281301,a_a000012,capital,2022-08-10,00:38:00
2054,6037b804983c662d,127.018102,37.281301,a_a000012,capital,2022-08-10,00:39:00
2055,6037b804983c662d,127.018102,37.281301,a_a000012,capital,2022-08-10,00:42:00
2056,6037b804983c662d,127.018102,37.281301,a_a000012,capital,2022-08-10,00:43:00
...,...,...,...,...,...,...,...
61329660,8F7B737D-D759-434E-A1EB-F38935BBC75D,126.393435,33.249842,d_d001410,jeju,2022-09-09,22:59:00
61329661,8F7B737D-D759-434E-A1EB-F38935BBC75D,126.393435,33.249842,d_d001410,jeju,2022-09-09,23:00:00
61329662,8F7B737D-D759-434E-A1EB-F38935BBC75D,126.393370,33.250120,d_d001410,jeju,2022-09-09,23:07:00
61329663,8F7B737D-D759-434E-A1EB-F38935BBC75D,126.393337,33.250033,d_d001410,jeju,2022-09-09,23:08:00


In [ ]:
# 코드 데이터의 ins_dt 컬럼도 날짜로 봐야할까?
filtered_dfs_22["tc_codea_코드A"]

,idx,cd_nm,cd_a,cd_memo,cd_memo2,del_flag,order_num,perm_write,perm_edit,perm_delete,ins_dt,edit_dt,REGION
0,5,여행상태,STA,NaN,NaN,N,999,N,N,N,2022-07-05 10:33:02.000,NaN,capital
1,6,방문지유형코드,VIS,NaN,NaN,N,999,N,N,N,2022-07-05 10:45:53.000,NaN,capital
2,7,주요 이동수단,MOV,NaN,NaN,N,999,N,N,N,2022-07-05 10:48:39.000,NaN,capital
3,8,재방문여부,REV,NaN,NaN,N,999,N,N,N,2022-07-05 11:27:33.000,NaN,capital
4,9,방문선택이유,REN,NaN,NaN,N,999,N,N,N,2022-07-05 11:28:18.000,NaN,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,30,성별,GEN,NaN,NaN,N,999,N,N,N,2022-09-22 17:34:50.000,NaN,jeju
116,31,동반자연령대,AGE,NaN,NaN,N,999,N,N,N,2022-09-22 17:34:50.000,NaN,jeju
117,32,동반자동반상황,CST,NaN,NaN,N,999,N,N,N,2022-09-22 17:34:50.000,NaN,jeju
118,33,입장료구분,AMS,체험/입장및 관람활동 구분,NaN,N,999,N,N,N,2022-11-11 13:48:15.000,NaN,jeju


## 2. 데이터셋 저장

In [60]:
# 지역별 데이터 통합, 저장 함수 생성
def store_filtered_data(data, dir1="aihub", dir2="2022", is_gps="no", start_date="2022-06-01", end_date="2022-06-06"):
    # 기본 경로 설정 (data\aihub\2022)
    base_path = os.path.join("data", dir1, dir2)

    # gps 데이터가 아니라면
    if is_gps == "no":
        combine_path = os.path.join(base_path, "filtered", "region_data")
        # filtered 폴더 생성 (2022\filtered)
        os.makedirs(combine_path, exist_ok=True)

        for key in data.keys():
            print(f"{key} 데이터셋 저장 시작 \n저장 경로: {combine_path}\n")
            data[key].to_csv(os.path.join(combine_path, f"{key}.csv"), index=False)
    
        print(f"=================== {dir2} 필터링 데이터 {len(data.keys())}개 저장 완료 ({start_date} ~ {end_date}) ===================")
        # print(f"=================== {combine_path} 폴더 내 데이터 갯수: {len(os.listdir(combine_path))} ====================")

    else:
        combine_path = os.path.join(base_path, "filtered", "gps_data")
        # filtered 폴더 생성 (2022\filtered)
        os.makedirs(combine_path, exist_ok=True)
        file_name = "gps_" + f"{dir2}"
        data.to_parquet(os.path.join(combine_path, f"{file_name}.parquet"), engine="pyarrow", index=False)

        print(f"=================== {dir2} 필터링 데이터 {data.shape} 저장 완료 ({start_date} ~ {end_date}) ===================")

    # 기존 폴더가 있다면 삭제
    # if os.path.exists(combine_path):
    #     shutil.rmtree(combine_path)

    # filtered 폴더 생성 (2022\filtered)
    # os.makedirs(combine_path, exist_ok=True)

    

    # for key in data.keys():
    #     print(f"{key} 데이터셋 저장 시작 \n저장 경로: {combine_path}\n")
    #     data[key].to_csv(os.path.join(combine_path, f"{key}.csv"), index=False)
    

    print(f"=================== {combine_path} 폴더 내 데이터 갯수: {len(os.listdir(combine_path))} ====================")

In [56]:
store_filtered_data(filtered_dfs_22, is_gps="no", start_date="2022-08-10", end_date="2022-09-09")

tc_codea_코드A 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tc_codeb_코드B 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tc_sgg_시군구코드 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_activity_consume_his_활동소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_activity_his_활동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_adv_consume_his_사전소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_companion_info_동반자정보 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_lodge_consume_his_숙박소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_move_his_이동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_mvmn_consume_his_이동수단소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_tour_photo_관광사진 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_traveller_master_여행객 Master 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_travel_여행 데이터셋 저장 시작 
저장 경로: data\aihub\2022\fil

In [61]:
store_filtered_data(filtered_gps_dfs_22, is_gps="yes", start_date="2022-08-10", end_date="2022-09-09")

=================== 2022 필터링 데이터 (4267354, 7) 저장 완료 (2022-08-10 ~ 2022-09-09) ===================
=================== data\aihub\2022\filtered\gps_data 폴더 내 데이터 갯수: 1 ====================
